<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Scraping-linkedin-Posts" data-toc-modified-id="Scraping-linkedin-Posts-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Scraping linkedin Posts</a></span><ul class="toc-item"><li><span><a href="#Login" data-toc-modified-id="Login-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Login</a></span></li><li><span><a href="#Load-posts-page-&amp;-scroll-to-bottom" data-toc-modified-id="Load-posts-page-&amp;-scroll-to-bottom-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Load posts page &amp; scroll to bottom</a></span></li><li><span><a href="#Retrieve-data-from-loaded-page" data-toc-modified-id="Retrieve-data-from-loaded-page-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Retrieve data from loaded page</a></span></li></ul></li></ul></div>

# Scraping linkedin Posts

In [1]:
%pip install selenium
%pip freeze


[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
aiohttp==3.8.4
aiosignal==1.3.1
anyio==3.7.0
appnope==0.1.3
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
arrow==1.2.3
asttokens==2.2.1
async-timeout==4.0.2
attrs==23.1.0
backcall==0.2.0
beautifulsoup4==4.12.2
bleach==6.0.0
certifi==2023.5.7
cffi==1.15.1
charset-normalizer==3.1.0
click==8.1.3
comm==0.1.3
commonmark==0.9.1
contourpy==1.1.0
cycler==0.11.0
dataclasses-json==0.5.8
debugpy==1.6.7
decorator==5.1.1
defusedxml==0.7.1
distro==1.8.0
dnspython==2.3.0
email-validator==2.0.0.post2
exceptiongroup==1.1.1
executing==1.2.0
fastapi==0.97.0
fastjsonschema==2.17.1
fonttools==4.40.0
fqdn==1.5.1
frozenlist==1.3.3
greenlet==2.0.2
h11==0.14.0
httpcore==0.17.2
httptools==0.5.0
httpx==0.24.1
idna==3.4
importlib-metadata==6.7.0
importlib-resources==5.12.0
ipykernel==6.23.3
ipython==8.12.2
ipython-genutils==0.2.0
ipywid

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
import time
import datetime
import pandas as pd
import re as re
from importlib.metadata import version

In [3]:
PAGE = 'https://www.linkedin.com/company/mgm-technology-partners-gmbh'

In [4]:

try:
    f= open("credentials.txt","r")
    contents = f.read()
    username = contents.replace("=",",").split(",")[1]
    password = contents.replace("=",",").split(",")[3]
except:
    f= open("credentials.txt","w+")
    username = input('Enter your linkedin username: ')
    password = input('Enter your linkedin password: ')
    f.write("username={}, password={}".format(username,password))
    f.close()

In [5]:
print(username, password)

till@mgm-tp.com django123!


## Login

In [6]:
# Login

#access Webriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")
#chrome_options.add_argument("--headless=new")
browser = webdriver.Chrome(options=chrome_options)

#Open login page
browser.get('https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin')

#Enter login info:
elementID = browser.find_element(by=By.ID, value='username')   #.find_element_by_id('username')
elementID.send_keys(username)

elementID = browser.find_element(by=By.ID, value='password')#find_element_by_id('password')
elementID.send_keys(password)
#Note: replace the keys "username" and "password" with your LinkedIn login info
elementID.submit()


## Load posts page & scroll to bottom

In [7]:

# #Go to webpage
company_posts_page = PAGE + '/posts/'
print(company_posts_page)
browser.get(company_posts_page)


SCROLL_PAUSE_TIME = 1.5

# Get scroll height
last_height = browser.execute_script("return document.body.scrollHeight")
scroll_page = 0

while True:
    # Scroll down to bottom
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    scroll_page += 1
    print(f"Scrolling page {scroll_page}")
    
    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height


company_page = browser.page_source   


https://www.linkedin.com/company/mgm-technology-partners-gmbh/posts/
Scrolling page 1
Scrolling page 2
Scrolling page 3
Scrolling page 4
Scrolling page 5
Scrolling page 6
Scrolling page 7
Scrolling page 8
Scrolling page 9
Scrolling page 10
Scrolling page 11
Scrolling page 12
Scrolling page 13
Scrolling page 14
Scrolling page 15
Scrolling page 16
Scrolling page 17


## Retrieve data from loaded page

In [8]:
linkedin_soup = bs(company_page.encode("utf-8"), "html")

# Write it to disk for analyzing
f= open("linkedin_soup.html","w+")
f.write(linkedin_soup.prettify())
f.close()

In [16]:
containers = linkedin_soup.find_all(class_="occludable-update")
#container = containers[0].find("div","display-flex feed-shared-actor display-flex feed-shared-actor--with-control-menu ember-view")

print(f"Number of container: {len(containers)}")
#print(containers[0].prettify())

# Write it to disk for analyzing
f= open("container_0.html","w+")
f.write(containers[0].prettify())
f.close()

Number of container: 147


In [10]:
post_dates = []
post_texts = []
post_likes = []
post_comments = []
video_views = []
media_links = []
media_type = []

for container in containers:

    try:
        posted_date = container.find("span",{"class":"visually-hidden"})
        text_box = container.find("div",{"class":"feed-shared-update-v2__description-wrapper"})
        text = text_box.find("span",{"dir":"ltr"})
        new_likes = container.findAll("li", {"class":"social-details-social-counts__reactions social-details-social-counts__item"})
        new_comments = container.findAll("li", {"class": "social-details-social-counts__comments social-details-social-counts__item"})


        post_dates.append(posted_date.text.strip())
        post_texts.append(text.text.strip())



        try:
            video_box = container.findAll("div",{"class": "feed-shared-update-v2__content feed-shared-linkedin-video ember-view"})
            video_link = video_box[0].find("video", {"class":"vjs-tech"})
            media_links.append(video_link['src'])
            media_type.append("Video")
        except:
            try:
                image_box = container.findAll("div",{"class": "feed-shared-image__container"})
                image_link = image_box[0].find("img", {"class":"ivm-view-attr__img--centered feed-shared-image__image feed-shared-image__image--constrained lazy-image ember-view"})
                media_links.append(image_link['src'])
                media_type.append("Image")
            except:
                try:
                    #mutiple shared images
                    image_box = container.findAll("div",{"class": "feed-shared-image__container"})
                    image_link = image_box[0].find("img", {"class":"ivm-view-attr__img--centered feed-shared-image__image lazy-image ember-view"})
                    media_links.append(image_link['src'])
                    media_type.append("Multiple Images")
                except:
                    try:
                        article_box = container.findAll("div",{"class": "feed-shared-article__description-container"})
                        article_link = article_box[0].find('a', href=True)
                        media_links.append(article_link['href'])
                        media_type.append("Article")
                    except:
                        try:
                            video_box = container.findAll("div",{"class": "feed-shared-external-video__meta"})          
                            video_link = video_box[0].find('a', href=True)
                            media_links.append(video_link['href'])
                            media_type.append("Youtube Video")   
                        except:
                            try:
                                poll_box = container.findAll("div",{"class": "feed-shared-update-v2__content overflow-hidden feed-shared-poll ember-view"})
                                media_links.append("None")
                                media_type.append("Other: Poll, Shared Post, etc")
                            except:
                                media_links.append("None")
                                media_type.append("Unknown")



        #Getting Video Views. (The folling three lines prevents class name overlap)
        view_container2 = set(container.findAll("li", {'class':["social-details-social-counts__item"]}))
        view_container1 = set(container.findAll("li", {'class':["social-details-social-counts__reactions","social-details-social-counts__comments social-details-social-counts__item"]}))
        result = view_container2 - view_container1

        view_container = []
        for i in result:
            view_container += i

        try:
            video_views.append(view_container[1].text.strip().replace(' Views',''))

        except:
            video_views.append('N/A')


        try:
            post_likes.append(new_likes[0].text.strip())
        except:
            post_likes.append(0)
            pass

        try:
            post_comments.append(new_comments[0].text.strip())                           
        except:                                                           
            post_comments.append(0)
            pass
    
    except:
        pass



In [11]:
post_dates
post_texts

['Die Implementierung moderner Register erfordert eine umfassende Analyse und die Nutzung vorhandener Standards. In unserem Blogbeitrag besprechen wir die Komplexität der Registerlandschaft und die Herausforderungen bei der Implementierung moderner Register sowie Lösungsansätze.Wie eine vollumfassende Analyse, die Nutzung von Standards und vorhandenen Strukturen sowie der Einsatz von Low Code-Plattformen dabei helfen können, fachliche und technische Herausforderungen effizient zu bewältigen, erfahren Sie hier:➡️ https://lnkd.in/eGZ8PVhP Julie Wayaffe, Dietmar Schmidt, Janos S.#Registermodernisierung #Digitalisierung #LowCode #öffentlicheverwaltung',
 '"Als ich hier angefangen habe, kannte ich mich weder mit Programmiersprachen noch mit den Tools aus, die hier angewandt werden." Unser Kollege Ricardo Bosin erzählt Dir, wie sein Entwickler-Leben bei mgm technology partners aussieht und wie wir ihn im Vorstellungsgespräch für uns gewinnen konnten. ➡️ Link zum Podcast: https://shorturl.at/

In [12]:
post_texts[0]

'Die Implementierung moderner Register erfordert eine umfassende Analyse und die Nutzung vorhandener Standards. In unserem Blogbeitrag besprechen wir die Komplexität der Registerlandschaft und die Herausforderungen bei der Implementierung moderner Register sowie Lösungsansätze.Wie eine vollumfassende Analyse, die Nutzung von Standards und vorhandenen Strukturen sowie der Einsatz von Low Code-Plattformen dabei helfen können, fachliche und technische Herausforderungen effizient zu bewältigen, erfahren Sie hier:➡️ https://lnkd.in/eGZ8PVhP Julie Wayaffe, Dietmar Schmidt, Janos S.#Registermodernisierung #Digitalisierung #LowCode #öffentlicheverwaltung'

In [13]:
articles = linkedin_soup.find_all("article")
print (len(articles))

66


In [14]:
# Looking for the relative date (in d, w, mo, yr)
# It has the shape: "1yr •"
p = re.compile(r'\d{1,2}(d|w|mo|yr)\s•')
m = re.compile(r'\d{1,2}(d|w|mo|yr)\s•').search(containers[4].prettify())
dateHumanReadable = ""
if m:
    dateHumanReadable = m.group()
    print('Match found: ', dateHumanReadable)
else:
    print('No match')

Match found:  4d •


In [15]:
# Transform "6d •" to a proper datetime
def linkedinRelDate2datetime (relative_date): 
    p = re.compile('\d{1,2}')
    m = p.search(relative_date)
    amount = float(m.group())
    p = re.compile('(d|w|mo|yr)')
    m = p.search(relative_date)
    unit = m.group()
    if unit == 'yr':
        amount *= 365
    elif unit == 'mo':
        amount *= 30
    elif unit == 'w':
        amount *= 7
    print(f"Amount in days: {amount}")
    # Calculate the date from today's, and return it
    howRecent = datetime.timedelta(days=amount)
    todaysDate = datetime.datetime.today()
    date = (todaysDate - howRecent).replace(hour=0,minute=0,second=0,microsecond=0)
    return date
    
#print(linkedinRelDate2datetime("1yr •"))